In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib.pyplot import imshow

%store -r IMAGE_WIDTH
%store -r IMAGE_HEIGHT
%store -r MAX_CAPTCHA
%store -r CHAR_SET_LEN
%store -r charSet

D:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def GetImageWH(image):
    import numpy as np
    
    if isinstance(image, str):    
        from PIL import Image
        image = Image.open(image)
        
    img_np = np.array(image)
    
    return img_np.shape[1], img_np.shape[0]

In [3]:
print(GetImageWH('1.jpg'))

(90, 25)


In [4]:
# #define possible alphabets, numbers
# numbers = ['0','1','2','3','4','5','6','7','8','9']
# alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
# ALPHABETs = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

# # Define max captcha = 4 digits
# MAX_CAPTCHA = 4 

# # Define Image Properties
# IMAGE_HEIGHT = 60
# IMAGE_WIDTH = 160

# # Define char set length
# charset = numbers + alphabets + ALPHABETs
# CHAR_SET_LEN = len(charset)
# print(CHAR_SET_LEN)

In [5]:
def RandomCharSet(charCount = 4):
    import random as rand
#     import ipynb.fs.full.Definitions
    
    %store -r charSet
    
    charChoices = []
    
    for i in range(charCount):
        charChoices.append(str(rand.choice(charSet)))
    
    return charChoices

In [6]:
import random
print(RandomCharSet(4))
# print(sys.path)

['6', '3', 'X', 'T']


In [7]:
def GetCaptchaTextAndImage(charCount = 4):
    from captcha.image import ImageCaptcha
    import numpy as np
    from PIL import Image
    
    captcha = ImageCaptcha()
    charSet = RandomCharSet(charCount)
    image = captcha.generate(charSet)
    image = Image.open(image)
    image = np.array(image)
    
    return charSet, image


In [8]:
# import ipynb.fs.full.Create_Image
# from ipynb.fs.full.Create_Image import GetCaptchaTextAndImage
from PIL import Image
from matplotlib.pyplot import imshow

text, image = GetCaptchaTextAndImage()

print(text)

print(type(image))

# imshow(image)
# print(image.shape)
# print(GetImageWH(image))

['K', '7', 'W', 'S']
<class 'numpy.ndarray'>


In [9]:
# 灰階化
def Convert2gray(image):
	if len(image.shape) > 2:
		gray = np.mean(image, -1)
		return gray
	else:
		return image

In [26]:
# Normalize Text To Vector
def Text2Vector(text):
#     import ipynb.fs.full.Definitions
    
    text_len = len(text)
    if text_len > MAX_CAPTCHA:
        raise ValueError('MAX_CAPTCHA out of length')

    vector = np.zeros(MAX_CAPTCHA*CHAR_SET_LEN)
    
    def GetCharIndex(char):
        return charSet.index(char)
    
    for index, char in enumerate(text):
        idx = index * CHAR_SET_LEN + GetCharIndex(char)
        vector[idx] = 1
        
    return vector

In [27]:
# vec = Text2Vector('hC2a')
# print(vec.nonzero()[0])

def Vector2Text(vector):
    text = []
    charPositions = vec.nonzero()[0]
    
    for index, charPos in enumerate(charPositions):
        charIndex = charPos - index * CHAR_SET_LEN
        text.append(charSet[charIndex])
    
    return text

In [28]:
def GenerateNextBatch(batchSize=128):
    import numpy as np
    
    batch_x = np.zeros([batchSize, IMAGE_HEIGHT*IMAGE_WIDTH])
    batch_y = np.zeros([batchSize, MAX_CAPTCHA*CHAR_SET_LEN])

    # Make sure image is 160(W) * 60(H) * 3(RGB) format
    def GetWrappedCaptchaTextAndImage():
#         from ipynb.fs.full.Create_Image import GetCaptchaTextAndImage
        
        while True:
            text, image = GetCaptchaTextAndImage()
            if image.shape == (60, 160, 3):#此部分应该与开头部分图片宽高吻合
                return text, image

    for i in range(batchSize):
        text, image = GetWrappedCaptchaTextAndImage()
        image = Convert2gray(image)

        # 将图片数组一维化 同时将文本也对应在两个二维组的同一行
        batch_x[i,:] = image.flatten() / 255 # (image.flatten()-128)/128  mean为0
        batch_y[i,:] = Text2Vector(text)

    # 返回该训练批次
    return batch_x, batch_y

In [29]:
print(GenerateNextBatch())

(array([[0.96470588, 0.96470588, 0.96470588, ..., 0.96470588, 0.96470588,
        0.96470588],
       [0.97385621, 0.97385621, 0.97385621, ..., 0.97385621, 0.97385621,
        0.97385621],
       [0.9620915 , 0.9620915 , 0.9620915 , ..., 0.9620915 , 0.9620915 ,
        0.9620915 ],
       ...,
       [0.97777778, 0.97777778, 0.97777778, ..., 0.97777778, 0.97777778,
        0.97777778],
       [0.97385621, 0.97385621, 0.97385621, ..., 0.97385621, 0.97385621,
        0.97385621],
       [0.96862745, 0.96862745, 0.96862745, ..., 0.96862745, 0.96862745,
        0.96862745]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]))


In [30]:
X = tf.placeholder(tf.float32, [None, IMAGE_HEIGHT*IMAGE_WIDTH])
Y = tf.placeholder(tf.float32, [None, MAX_CAPTCHA*CHAR_SET_LEN])
keep_prob = tf.placeholder(tf.float32) # dropout

In [31]:
# Define CNN
def crack_captcha_cnn(w_alpha=0.01, b_alpha=0.1):
    # 将占位符 转换为 按照图片给的新样式
    x = tf.reshape(X, shape=[-1, IMAGE_HEIGHT, IMAGE_WIDTH, 1])

    #w_c1_alpha = np.sqrt(2.0/(IMAGE_HEIGHT*IMAGE_WIDTH)) #
    #w_c2_alpha = np.sqrt(2.0/(3*3*32))
    #w_c3_alpha = np.sqrt(2.0/(3*3*64))
    #w_d1_alpha = np.sqrt(2.0/(8*32*64))
    #out_alpha = np.sqrt(2.0/1024)

    # 3 conv layer
    w_c1 = tf.Variable(w_alpha*tf.random_normal([3, 3, 1, 32])) # 从正太分布输出随机值
    b_c1 = tf.Variable(b_alpha*tf.random_normal([32]))
    conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(x, w_c1, strides=[1, 1, 1, 1], padding='SAME'), b_c1))
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv1 = tf.nn.dropout(conv1, keep_prob)

    w_c2 = tf.Variable(w_alpha*tf.random_normal([3, 3, 32, 64]))
    b_c2 = tf.Variable(b_alpha*tf.random_normal([64]))
    conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv1, w_c2, strides=[1, 1, 1, 1], padding='SAME'), b_c2))
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.dropout(conv2, keep_prob)

    w_c3 = tf.Variable(w_alpha*tf.random_normal([3, 3, 64, 64]))
    b_c3 = tf.Variable(b_alpha*tf.random_normal([64]))
    conv3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv2, w_c3, strides=[1, 1, 1, 1], padding='SAME'), b_c3))
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv3 = tf.nn.dropout(conv3, keep_prob)

    # Fully connected layer
    w_d = tf.Variable(w_alpha*tf.random_normal([8*20*64, 1024]))
    b_d = tf.Variable(b_alpha*tf.random_normal([1024]))
    dense = tf.reshape(conv3, [-1, w_d.get_shape().as_list()[0]])
    dense = tf.nn.relu(tf.add(tf.matmul(dense, w_d), b_d))
    dense = tf.nn.dropout(dense, keep_prob)

    w_out = tf.Variable(w_alpha*tf.random_normal([1024, MAX_CAPTCHA*CHAR_SET_LEN]))
    b_out = tf.Variable(b_alpha*tf.random_normal([MAX_CAPTCHA*CHAR_SET_LEN]))
    out = tf.add(tf.matmul(dense, w_out), b_out)
    #out = tf.nn.softmax(out)
    return out

In [32]:
def train_crack_captcha_cnn():
    output = crack_captcha_cnn()
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, Y))
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=Y))
    # 最后一层用来分类的softmax和sigmoid有什么不同？
    # optimizer 为了加快训练 learning_rate应该开始大，然后慢慢衰
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    predict = tf.reshape(output, [-1, MAX_CAPTCHA, CHAR_SET_LEN])
    max_idx_p = tf.argmax(predict, 2)
    max_idx_l = tf.argmax(tf.reshape(Y, [-1, MAX_CAPTCHA, CHAR_SET_LEN]), 2)
    correct_pred = tf.equal(max_idx_p, max_idx_l)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        step = 0
        while True:
            batch_x, batch_y = GenerateNextBatch(64)
            _, loss_ = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.75})
            print(step, loss_)

            # 每100 step计算一次准确率
            if step % 100 == 0:
                batch_x_test, batch_y_test = GenerateNextBatch(100)
                acc = sess.run(accuracy, feed_dict={X: batch_x_test, Y: batch_y_test, keep_prob: 1.})
                print(step, acc)
                # 如果准确率大于50%,保存模型,完成训练
                if acc > 0.7:
                    saver.save(sess, "crack_capcha.model", global_step=step)
                    break
            step += 1

In [ ]:
train_crack_captcha_cnn()

0 0.6866401
0 0.0175
1 0.56271255
2 0.32768753
3 0.116980836
4 0.09543078
5 0.14326423
6 0.18288644
7 0.1987783
8 0.19781256
9 0.18298851
10 0.16309252
11 0.1403823
12 0.11629746
13 0.10215574
14 0.09243608
15 0.09511829
16 0.10445235
17 0.1105131
18 0.10641163
19 0.099039696
20 0.09270748
21 0.0866262
22 0.08510802
23 0.08434649
24 0.08676955
25 0.08881244
26 0.09053704
27 0.09085564
28 0.08875621
29 0.090805456
30 0.08736399
31 0.088841714
32 0.08665676
33 0.088257775
34 0.0881567
35 0.08678137
36 0.087650485
37 0.08635847
38 0.0855105
39 0.08499701
40 0.08451037
41 0.084284246
42 0.08380719
43 0.08519139
44 0.08431613
45 0.08421059
46 0.08473195
47 0.08479389
48 0.08424851
49 0.08456392
50 0.08321936
51 0.08409954
52 0.08460499
53 0.08490343
54 0.08511301
55 0.08511762
56 0.083990805
57 0.08469956
58 0.084201075
59 0.08348214
60 0.08404818
61 0.08423814
62 0.08364153
63 0.08464364
64 0.08431364
65 0.08424005
66 0.08386927
67 0.08443513
68 0.08398487
69 0.0840735
70 0.08373485
71 0.0

545 0.08301769
546 0.08356937
547 0.08359177
548 0.084048994
549 0.08322386
550 0.08379256
551 0.08310815
552 0.083456166
553 0.08363953
554 0.08326316
555 0.08369629
556 0.0834887
557 0.08358735
558 0.083674915
559 0.083206184
560 0.08357269
561 0.083478644
562 0.08367217
563 0.083015166
564 0.08345178
565 0.08342552
566 0.08300889
567 0.083032474
568 0.08376707
569 0.08364705
570 0.08326602
571 0.08353613
572 0.0833978
573 0.08355791
574 0.08340158
575 0.083852135
576 0.08339947
577 0.08306447
578 0.083484635
579 0.08364915
580 0.08327292
581 0.08282259
582 0.08324402
583 0.08339391
584 0.08360209
585 0.08298487
586 0.08327268
587 0.08346831
588 0.08344033
589 0.08305988
590 0.08320447
591 0.08393342
592 0.08326478
593 0.08332265
594 0.0833304
595 0.08316163
596 0.08376026
597 0.08338142
598 0.083440766
599 0.08353056
600 0.083605446
600 0.02
601 0.08346036
602 0.083478406
603 0.08304024
604 0.08357434
605 0.08365244
606 0.08321482
607 0.08339507
608 0.08320891
609 0.08356193
610 0.0